# Forecasting respiratory admissions using ARIMA and Prophet

> Respiratory illnesses are **highly** seasonal.  As people with respiratory conditions are more susceptable to illness in Winter and there is a higher circulation of viruses, such as influenza, hospitals generally experience far higher demand in this period of the year.

**In this case study notebook:**

* You will work with a daily level respiratory dataset and produce an 84 day (12 week) forecast ahead.
* You will need to check that our selected method is better than a Naive benchmark.

**You will learn:**

* How to apply ARIMA and Prophet models and to select the most promising model using a holdout sample.

# Setup:

## Standard imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## forecast-tools

In [ ]:
from forecast_tools.model_selection import auto_naive
from forecast_tools.metrics import coverage, mean_absolute_error

## Importing ARIMA and Prophet

In [ ]:
# arima
from pmdarima import auto_arima, ARIMA
import pmdarima as pm

#should be version 1.6.0
pm.__version__

In [ ]:
# prophet 
from prophet import Prophet
import prophet

#should be version 1.0+
prophet.__version__

# Exercise 1: load and train test split respiratory admissions daily dataset

The first split we will do in summer 2018.  Exercuise 4 will look at a winter split.

**Task:**

* Load the `resp_admissions.csv` dataset
* Use the `date` column as your DateTimeIndex
* Perform a train-test split holding back 84 days of data as a test set.
* Plot the training data
* Perform a seasonal decomposition

In [ ]:
#example solution

admits = pd.read_csv('../../hds_data/resp_admissions.csv', parse_dates=True, 
                     index_col='date', dayfirst=True, dtype='int')

admits.index.freq = 'D'

In [ ]:
admits.info()

In [ ]:
# train test split
holdout = 84
training_length = len(admits) - holdout

train = admits.iloc[:training_length]
test = admits.iloc[training_length:]

In [ ]:
train.plot(figsize=(12,4))

# Exercise 2: Select a Naive forecasting benchmark

**Task:** 
    
* Select a Naive method to serve as a benchmark for ARIMA and Prophet.
* Using a simple holdout sample calculate the MAE

In [ ]:
# your code here ...

In [ ]:
naive_model = auto_naive(train, seasonal_period=7)
naive_model

In [ ]:
naive_preds = naive_model['model'].fit_predict(train, horizon=len(test))
mean_absolute_error(naive_preds, test)

# Exercise 3: Modelling the series using an ARIMA 

Let's first model the series using an autoARIMA model.

## Exercise 3.1. Use `auto_arima` to select ARIMA parameters

**Task**:

* Select and fit an ARIMA model.
* What model was selected?

In [ ]:
#your code here ...

In [ ]:
# example solution

# search for best model using auto_arima
# arima_model = auto_arima(train, m=7, suppress_warnings=True)

# best model = (3, 1, 2)(1, 0, 1, 7)
arima_model = ARIMA(order=(3, 1, 2), seasonal_order=(1, 0, 1, 7), 
                    suppress_warnings=True)
arima_model.fit(train)

In [ ]:
# take a look at the best model.

arima_model.summary()

## Exercise 3.2 Use the ARIMA model to predict the test set in summer

**Task**:
   
* Use your preferred ARIMA model to predict 84 days ahead
* Produce a mean point forecast and a 95% prediction interval
* Plot the predictions
* Calculate the Mean Absolute Error of the prediction.
* Calculate the Prediction Interval Coverage

In [ ]:
#your code here ...

In [ ]:
h = 84
forecast = arima_model.predict(n_periods=h, alpha=0.05, return_conf_int=True)
forecast = pd.concat([pd.DataFrame(forecast[0]), pd.DataFrame(forecast[1])], 
                     axis=1)
forecast.columns=['mean', 'lower_95', 'upper_95']
idx = pd.date_range(start=train.index[-1], periods=h+1, freq='D')[1:]
forecast.index = idx
forecast

In [ ]:
# plot the mean forecast and 95% prediction interval

fig, ax = plt.subplots(1, 1, figsize=(12,4))
ax.plot(train['resp_admits'].iloc[-90:])
ax.plot(forecast['mean'])
ax.plot(test['resp_admits'])

ax.fill_between(forecast.index, forecast['mean'], forecast['upper_95'],
                alpha=0.2,
                label='95% PI', color='purple');

ax.fill_between(forecast.index, forecast['mean'], forecast['lower_95'],
                alpha=0.2,
                label='95% PI', color='purple');

fig.legend(['train', 'point forecast', 'test', '_ignore', '95% PI'], 
           loc='upper center', ncol=4);

In [ ]:
# Calculate MAE

from pmdarima.metrics import smape

rmse_arima = mean_absolute_error(forecast['mean'], test['resp_admits'])
smape_arima = smape(forecast['mean'], test['resp_admits'])

print(rmse_arima)
print(smape_arima)

# Exercise 4: Predict the time series using Prophet

## Exercise 4.1: Wrangle the data into Prophet format

Prophet requires the data to be in a `pd.DataFrame` with two columns: `ds` for date and `y` for the observations.

**Task**: 

* Create a new `pd.DataFrame` for the respiratory admissions data.  
* The data should be in the format suitable for `Prophet` models.

**Hints:**
* Create a copy of your original data
* It might be useful to write a function to do the formatting.  This means you can easily reuse your code in the future.


In [ ]:
# your code here ...

In [ ]:
def prophet_formatted_df(df):
    '''
    Convert a pd.DataFrame with a DateTimeIndex into a prophet
    formatted dataframe
    
    Returns:
    ------
    pd.DataFrame[['ds', 'y']]
    '''
    df_prophet = df.copy()
    df_prophet = df_prophet.reset_index()
    df_prophet.columns = ['ds', 'y']
    return df_prophet
    
    

In [ ]:
train_proph = prophet_formatted_df(train)
train_proph.head()

# Exercise 4.2: Fit and predict with the prophet model

**Task**:
    
* Fit a `Prophet` model. (do not worry about holidays)
* The `interval_width` should be 0.95
* Predict 84 days into he future.
* Plot the components and the forecast
* Calculate the MAE of the Forecast
* Calculate the Prediction Interval Coverage

**Questions:**
* Do you think Prophet offers anything over and above the ARIMA model in this instance?
* If so what are concepts within Prophet that help?

In [ ]:
#your code here ...

In [ ]:
# example solution

h = 84
model = Prophet(interval_width=0.95)
model.fit(train_proph)
future = model.make_future_dataframe(periods=h)
forecast = model.predict(future)

In [ ]:
forecast.tail()

In [ ]:
fig2 = model.plot_components(forecast)

In [ ]:
#plot the mean forecast and 95% prediction interval
f = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].iloc[-h:]
f.index = f['ds']
f.index.freq = 'D'
#f.drop(['ds'], inplace=True)

fig, ax = plt.subplots(1, 1, figsize=(12,4))
ax.plot(train['resp_admits'].iloc[-90:])
ax.plot(f['yhat'])
ax.plot(test['resp_admits'].iloc[:h])

ax.fill_between(f.index, f['yhat'], f['yhat_upper'],
                alpha=0.2,
                label='95% PI', color='purple');

ax.fill_between(f.index, f['yhat'], f['yhat_lower'],
                alpha=0.2,
                label='95% PI', color='purple');

fig.legend(['train', 'point forecast', 'test', '_ignore', '95% PI'], 
           loc='upper center', ncol=4);

In [ ]:
rmse_arima = mean_absolute_error(f['yhat'], test['resp_admits'].iloc[:h])
smape_arima = smape(f['yhat'], test['resp_admits'].iloc[:h])

print(rmse_arima)
print(smape_arima)

# Exercise 5: Forecasting just before the start of Winter

Now let's assume we need to make a forecast from just before the onset of winter.

**Task:**

* Train test split using a holdout sample of 245 days (test will begin from late November 2017).
* Fit an ARIMA(1, 0, 2)x(1, 0, 1, 7) and a Prophet model to the data.
* Make a 84 day prediction.  
* Include 95% prediction intervals
* Plot the results.
* Calculate MAE and Coverage for each forecast at 7 day intervals up to 84 days.  I.e. 7, 14, 28, ..., 84 

**Hints:**
* The ARIMA model was selected using `auto_arima`.  Feel free to check or try a different model. One option would be difference the time series.

**Questions**:
* What would you say the key differences between the models are?
* Which forecast method would you choose and why?
* How do the forecasts compare to a Naive model?

In [ ]:
# your code goes here ...

In [ ]:
# example solution

#train test split
holdout = 245
training_length = len(admits) - holdout

train = admits.iloc[:training_length]
test = admits.iloc[training_length:]
train.tail()

In [ ]:
#ARIMA

#auto select via AIC
#arima_model = auto_arima(train, m=7, suppress_warnings=True, 
#                         error_action="ignore")

arima_model = ARIMA(order=(1,0,2), seasonal_order=(1, 0, 1, 7), 
                    suppress_warnings=True)
arima_model.fit(train)
arima_model.summary()

In [ ]:
h = 84
arima_forecast = arima_model.predict(n_periods=h, alpha=0.05, return_conf_int=True)
arima_forecast = pd.concat([pd.DataFrame(arima_forecast[0]), 
                            pd.DataFrame(arima_forecast[1])], 
                            axis=1)
arima_forecast.columns=['mean', 'lower_95', 'upper_95']
idx = pd.date_range(start=train.index[-1], periods=h+1, freq='D')[1:]
arima_forecast.index = idx
arima_forecast.head()

In [ ]:
train_proph = prophet_formatted_df(train)

h = 84
model = Prophet(interval_width=0.95)
model.fit(train_proph)
future = model.make_future_dataframe(periods=h)
forecast = model.predict(future)

In [ ]:
#plot the mean forecast and 95% prediction interval
f = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].iloc[-h:]
f.index = f['ds']
f.index.freq = 'D'
#f.drop(['ds'], inplace=True)

fig, ax = plt.subplots(2, 1, figsize=(12,8), sharex=True)

#plot ARIMA
ax[0].plot(train['resp_admits'].iloc[-90:])
ax[0].plot(arima_forecast['mean'])
ax[0].plot(test['resp_admits'].iloc[:h])

ax[0].fill_between(arima_forecast.index, arima_forecast['mean'], 
                   arima_forecast['upper_95'],
                   alpha=0.2,
                   label='95% PI', color='purple');

ax[0].fill_between(arima_forecast.index, 
                   arima_forecast['mean'], 
                   arima_forecast['lower_95'],
                   alpha=0.2,
                   label='95% PI', color='purple');

#plot Prophet
ax[1].plot(train['resp_admits'].iloc[-90:])
ax[1].plot(f['yhat'])
ax[1].plot(test['resp_admits'].iloc[:h])

ax[1].fill_between(f.index, f['yhat'], f['yhat_upper'],
                alpha=0.2,
                label='95% PI', color='purple');

ax[1].fill_between(f.index, f['yhat'], f['yhat_lower'],
                alpha=0.2,
                label='95% PI', color='purple');


fig.legend(['train', 'point forecast', 'test', '_ignore', '95% PI'], 
           loc='lower center', ncol=4);

In [ ]:
#calculate MAE and coverage at 7 days intervals during forecast period.
h = 84
horizons = [i for i in range(7, h+7, 7)]

arima_results = []
prophet_results = []

arima_coverage = []
prophet_coverage = []

for h in horizons:
    #mae
    m1_mae = mean_absolute_error(arima_forecast['mean'].iloc[:h], 
                                 test['resp_admits'].iloc[:h])
    m2_mae = mean_absolute_error(f['yhat'].iloc[:h], 
                                 test['resp_admits'].iloc[:h])
    
    #coverage
    m1_cov = coverage(test['resp_admits'].iloc[:h], 
                      arima_forecast[['lower_95', 'upper_95']].iloc[:h].to_numpy())
    
    m2_cov = coverage(test['resp_admits'].iloc[:h], 
                      f[['yhat_lower', 'yhat_upper']].iloc[:h].to_numpy())
    
    arima_results.append(m1_mae)
    prophet_results.append(m2_mae)
    
    arima_coverage.append(m1_cov)
    prophet_coverage.append(m2_cov)

fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
ax[0].plot(horizons, arima_results, label='ARIMA')
ax[0].plot(horizons, prophet_results, label='Prophet')

ax[0].set_title('MAE')

ax[1].plot(horizons, arima_coverage, label='ARIMA')
ax[1].plot(horizons, prophet_coverage, label='Prophet')
ax[1].axhline(y=0.95, color='r', linestyle='--')

ax[1].set_xlabel('horizon (days)')
ax[1].set_title('coverage')
fig.legend(['ARIMA', 'Prophet'], ncol=2, loc='upper center')

# Exercise 5: Adding in holidays to Prophet

**Task:**

* Add in Prophet's default holidays to test and refit the model
* Explore the impact of holidays.
* Use MAE and Coverage to compare the performance to the original version.


**Hints:**

* Prophet has built in routines to help.

```python
model.add_country_holidays(country_name='England')
```

**Questions**
* Do you need all of the UK holidays?
* Do the holiday effects make sense in the different seasons?
* Is there a big impact on forecasting performance overall or is limited to the holidays?

In [ ]:
#your code here...

In [ ]:
#example solution

train_proph = prophet_formatted_df(train)

h = 84
model = Prophet(interval_width=0.95, changepoint_prior_scale=0.3)
model.add_country_holidays(country_name='England')
model.fit(train_proph)
future_with_holidays = model.make_future_dataframe(periods=h)
forecast_holidays = model.predict(future_with_holidays)

In [ ]:
#these are the holidays included by default
model.train_holiday_names.to_list()

In [ ]:
model.plot_components(forecast_holidays);

In [ ]:
from fbprophet.plot import plot_forecast_component

#christmas day
plot_forecast_component(model, forecast_holidays, 'Christmas Day');

In [ ]:
#New Year's Day - probably not useful
plot_forecast_component(model, forecast_holidays, "New Year's Day");

In [ ]:
plot_forecast_component(model, forecast_holidays, "Spring Bank Holiday");

In [ ]:
#plot the mean forecast and 95% prediction interval

#just predictions
f2 = forecast_holidays[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].iloc[-h:]
f2.index = f2['ds']
f2.index.freq = 'D'
#f.drop(['ds'], inplace=True)

fig, ax = plt.subplots(2, 1, figsize=(12,8), sharex=True)

#plot Prophet with holidays
ax[0].plot(train['resp_admits'].iloc[-90:])
ax[0].plot(f2['yhat'])
ax[0].plot(test['resp_admits'].iloc[:h])

ax[0].fill_between(f2.index, f2['yhat'], f2['yhat_upper'],
                alpha=0.2,
                label='95% PI', color='purple');

ax[0].fill_between(f2.index, f2['yhat'], f2['yhat_lower'],
                alpha=0.2,
                label='95% PI', color='purple');



#plot Prophet 
ax[1].plot(train['resp_admits'].iloc[-90:])
ax[1].plot(f['yhat'])
ax[1].plot(test['resp_admits'].iloc[:h])

ax[1].fill_between(f.index, f['yhat'], f['yhat_upper'],
                alpha=0.2,
                label='95% PI', color='purple');

ax[1].fill_between(f.index, f['yhat'], f['yhat_lower'],
                alpha=0.2,
                label='95% PI', color='purple');


fig.legend(['train', 'point forecast', 'test', '_ignore', '95% PI'], 
           loc='lower center', ncol=4);

In [ ]:
#calculate MAE and coverage at 7 days intervals during forecast period.
h = 84
horizons = [i for i in range(7, h+7, 7)]

holiday_results = []
prophet_results = []

holiday_coverage = []
prophet_coverage = []

for h in horizons:
    #rmse
    m1_mae = mean_absolute_error(f2['yhat'].iloc[:h], 
                                 test['resp_admits'].iloc[:h])
    m2_mae = mean_absolute_error(f['yhat'].iloc[:h], 
                                 test['resp_admits'].iloc[:h])
    
    #coverage
    m1_cov = coverage(test['resp_admits'].iloc[:h], 
                      f2[['yhat_lower', 'yhat_upper']].iloc[:h].to_numpy())
    
    m2_cov = coverage(test['resp_admits'].iloc[:h], 
                      f[['yhat_lower', 'yhat_upper']].iloc[:h].to_numpy())
    
    holiday_results.append(m1_mae)
    prophet_results.append(m2_mae)
    
    holiday_coverage.append(m1_cov)
    prophet_coverage.append(m2_cov)

fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
ax[0].plot(horizons, holiday_results, label='ARIMA')
ax[0].plot(horizons, prophet_results, label='Prophet')

ax[0].set_title('MAE')

ax[1].plot(horizons, prophet_coverage, label='ARIMA')
ax[1].plot(horizons, prophet_coverage, label='Prophet')
ax[1].axhline(y=0.95, color='r', linestyle='--')

ax[1].set_xlabel('horizon (days)')
ax[1].set_title('coverage')
fig.legend(['Prophet with Holidays', 'Prophet no tuning'], 
           ncol=2, loc='upper center')

# Exercise 6: Forecasting respiratory admissions

**Task**
* Using all of the data and your selected model create a 84 day forecast.

In [ ]:
# your code here ...

In [ ]:
train_proph = prophet_formatted_df(admits)

h = 84
model = Prophet(interval_width=0.95)
model.add_country_holidays(country_name='England')
model.fit(train_proph)
future = model.make_future_dataframe(periods=h)
final_forecast = model.predict(future)

In [ ]:
#plot the mean forecast and 95% prediction interval

#just predictions
f3 = final_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].iloc[-h:]
f3.index = f3['ds']
f3.index.freq = 'D'


fig, ax = plt.subplots(1, 1, figsize=(12,8), sharex=True)

#plot Prophet with holidays
ax.plot(admits['resp_admits'].iloc[-365:])
ax.plot(f3['yhat'])

ax.fill_between(f3.index, f3['yhat'], f3['yhat_upper'],
                alpha=0.2,
                label='95% PI', color='purple');

ax.fill_between(f3.index, f3['yhat'], f3['yhat_lower'],
                alpha=0.2,
                label='95% PI', color='purple');

In [ ]:
# your code here ...

# Bonus Exercise 7:
**Task**
* Try a log transform of the data
* Does this improve forecast accuracy?

**Hints**
* Don't forget to back transform your predictions

In [ ]:
# your code here ...

In [ ]:
admit_ln = np.log(admits)

#train test split
holdout = 84
training_length = len(admits) - holdout

train_ln = admits.iloc[:training_length]
test_ln = admits.iloc[training_length:]

In [ ]:
#compare original and logged data

fig, ax = plt.subplots(2, 1, figsize=(12,8), sharex=True)
ax[0].plot(admits)
ax[1].plot(admit_ln)

In [ ]:
naive_model = auto_naive(train, seasonal_period=7)
naive_model

naive_preds = naive_model['model'].fit_predict(train_ln, horizon=len(test))
mean_absolute_error(naive_preds, test)

In [ ]:
#ARIMA

#auto select via AIC
#arima_model = auto_arima(train_ln, m=7, suppress_warnings=True, 
#                         error_action="ignore")

arima_model = ARIMA(order=(3,1,2), seasonal_order=(1, 0, 1, 7), 
                    suppress_warnings=True)
arima_model.fit(train_ln)
arima_model.summary()

In [ ]:
h = 84
arima_forecast = arima_model.predict(n_periods=h, alpha=0.05, 
                                     return_conf_int=True)
arima_forecast = pd.concat([pd.DataFrame(arima_forecast[0]), 
                            pd.DataFrame(arima_forecast[1])], 
                            axis=1)
arima_forecast.columns=['mean', 'lower_95', 'upper_95']
idx = pd.date_range(start=train_ln.index[-1], periods=h+1, freq='D')[1:]
arima_forecast.index = idx

mean_absolute_error(arima_forecast['mean'], test)

In [ ]:
#previous best Prophet

train_proph = prophet_formatted_df(train)

h = 84
model = Prophet(interval_width=0.95)
model.add_country_holidays(country_name='England')
model.fit(train_proph)
future = model.make_future_dataframe(periods=h)
best_forecast = model.predict(future)

In [ ]:
#Prophet
train_proph = prophet_formatted_df(train_ln)

h = 84
model = Prophet(interval_width=0.95)
model.add_country_holidays(country_name='England')
model.fit(train_proph)
future = model.make_future_dataframe(periods=h)
forecast_ln = model.predict(future)

In [ ]:
mean_absolute_error(forecast_ln['yhat'].iloc[-h:], test_ln)

In [ ]:
#plot the mean forecast and 95% prediction interval

#just predictions
f2 = forecast_ln[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].iloc[-h:]
f2.index = f2['ds']
f2.index.freq = 'D'

fig, ax = plt.subplots(2, 1, figsize=(12,8), sharex=True)

#plot ARIMA
ax[0].plot(train_ln['resp_admits'].iloc[-90:])
ax[0].plot(arima_forecast['mean'])
ax[0].plot(test_ln['resp_admits'].iloc[:h])

ax[0].fill_between(arima_forecast.index, arima_forecast['mean'], 
                   arima_forecast['upper_95'],
                   alpha=0.2,
                   label='95% PI', color='purple');

ax[0].fill_between(arima_forecast.index, 
                  arima_forecast['mean'], 
                   arima_forecast['lower_95'],
                   alpha=0.2,
                   label='95% PI', color='purple');

#plot Prophet 
ax[1].plot(train_ln['resp_admits'].iloc[-90:])
ax[1].plot(f2['yhat'])
ax[1].plot(test_ln['resp_admits'].iloc[:h])

ax[1].fill_between(f2.index, f2['yhat'], f2['yhat_upper'],
                alpha=0.2,
                label='95% PI', color='purple');

ax[1].fill_between(f2.index, f2['yhat'], f2['yhat_lower'],
                alpha=0.2,
                label='95% PI', color='purple');


fig.legend(['train', 'point forecast', 'test', '_ignore', '95% PI'], 
           loc='lower center', ncol=4);

In [ ]:
#calculate MAE and coverage at 7 days intervals during forecast period.
h = 84
horizons = [i for i in range(7, h+7, 7)]

f = forecast_ln[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].iloc[-h:]
f.index = f['ds']
f.index.freq = 'D'

f3 = best_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].iloc[-h:]
f3.index = f3['ds']
f3.index.freq = 'D'


arima_results = []
prophet_results = []
previous_best_results = []

arima_coverage = []
prophet_coverage = []
previous_best_coverage = []

for h in horizons:
    #mae
    
    #arima logged data
    m1_mae = mean_absolute_error(arima_forecast['mean'].iloc[:h], 
                                 test_ln['resp_admits'].iloc[:h])
    #prophet logged data
    m2_mae = mean_absolute_error(f['yhat'].iloc[:h], 
                                 test_ln['resp_admits'].iloc[:h])
    
    #from previous best model (unlogged data)
    m3_mae = mean_absolute_error(f3['yhat'].iloc[:h], 
                                 test['resp_admits'].iloc[:h])
    
    #coverage
    
    # arima logged
    m1_cov = coverage(test_ln['resp_admits'].iloc[:h], 
                      arima_forecast[['lower_95', 'upper_95']].iloc[:h].to_numpy())
    
    # prophet logged
    m2_cov = coverage(test_ln['resp_admits'].iloc[:h], 
                      f[['yhat_lower', 'yhat_upper']].iloc[:h].to_numpy())
    
    # prophet unlogged
    m3_cov = coverage(test['resp_admits'].iloc[:h], 
                      f3[['yhat_lower', 'yhat_upper']].iloc[:h].to_numpy())
    
    arima_results.append(m1_mae)
    prophet_results.append(m2_mae)
    previous_best_results.append(m3_mae)
    
    arima_coverage.append(m1_cov)
    prophet_coverage.append(m2_cov)
    previous_best_coverage.append(m3_cov)

fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
ax[0].plot(horizons, arima_results, label='ARIMA')
ax[0].plot(horizons, prophet_results, label='Prophet')
ax[0].plot(horizons, previous_best_results, label='Prophet unlogged data')

ax[0].set_title('MAE')

ax[1].plot(horizons, arima_coverage, label='ARIMA (logged data)')
ax[1].plot(horizons, prophet_coverage, label='Prophet (logged data)')
ax[1].plot(horizons, previous_best_coverage, label='Prophet unlogged data')
ax[1].axhline(y=0.95, color='r', linestyle='--')

ax[1].set_xlabel('horizon (days)')
ax[1].set_title('coverage')
fig.legend(['ARIMA', 'Prophet', 'Prophet unlogged'], 
           ncol=2, loc='upper center')

# Bonus Exercise 8: Cross-Validation

**Task**:
* Use time series cross validation to compare ARIMA, Prophet and the Naive model.